<b>HW 6: Character classification using KNN with PyTorch

Author:
</b> Brian Erichsen Fagundes


In [24]:
# Step 1: Data acquision + clenup
import pandas as pd
import numpy as np

# loads data into variable
data = pd.read_csv('ARIAL.csv')

# selects which columns to keep m_label and all the r{x} c{y}
columns_to_keep = ['m_label']
columns_to_keep += [f'r{r}c{c}' for r in range(0, 20) for c in range(0, 20)]
filtered_data = data[columns_to_keep]

# funtion that transforms dataframe returns 2 numpy arrays
# x sample x 20 x 20 has pixel val, y #samples x 1 array has ascii for each char
def transform_data(data_frame):
    # extract the pixel val and normalize data
    # . values converts from pandas to numpy array
    Xs = data_frame[[f'r{r}c{c}' for r in range(0, 20) for c in range(0, 20)]].values
    # makes it samples x 20 x 20 D / 256.0
    Xs = Xs.reshape(-1, 20, 20) / 256.0

    # extrac the ascii value for each char
    Ys = data_frame['m_label'].values
    # makes samples# x 1 Dim
    Ys = Ys.reshape(-1, 1)

    return Xs, Ys

Xs, Ys = transform_data(filtered_data)

# dictionary for label conversion - using set (collection of unique elements)
unique_chars = sorted(set(filtered_data['m_label']))
# maps each char to unique index
char_to_index = {char: idx for idx, char in enumerate(unique_chars)}
# maps each index back to char
index_to_char = {idx: char for char, idx in char_to_index.items()}

# convert labels to indices
Ys = np.array([char_to_index[char] for char in Ys.flatten()])

In [25]:
# Step 2: Build a Pytorch network where its archecture is
    # Convolution 2D layer (relu)
    # Max pooling layer
    # Convolution, another Max pooling
    # Dense layer (relu), dense layer
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import TensorDataset, DataLoader

# Tensor is numpy multi dim array
# Convert data to PyTorch tensors
Xs = torch.tensor(Xs, dtype=torch.float32).reshape(-1, 1, 20, 20) # between 0 and 1
Ys = torch.tensor(Ys, dtype=torch.long) # can be long int

# So we can iterate over batches
dataset = TensorDataset(Xs, Ys)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Network as a class with a constructor and forward method
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1d input, 6 outputs and 3 x 3 pixels kernel filter
        self.conv1 = nn.Conv2d(1, 6, 3)
        # kernel size of 2, reduces spatial dim by half, with stride of 2 for 2x2 kernel
        self.pool = nn.MaxPool2d(2, 2)
        # 6 from the 6 output layer in the 1st convolution layer
        self.conv2 = nn.Conv2d(6, 16, 3)
        # first dense layer has 16 * 3 * 3 input features and 120 neurons (output features)
        # after second pooling layer, we have 16 channels 3 x 3
        self.fc1 = nn.Linear(16 * 3 * 3, 120)
        #self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(120, len(unique_chars))

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))# conv1 -> relu -> max pool
        x = self.pool(F.relu(self.conv2(x)))# conv2 -> relu -> max pool
        x = x.view(-1, 16 * 3 * 3)# flattens the tensor back to 1 D
        x = F.relu(self.fc1(x)) # FC1 -> relu
        #x = F.relu(self.fc2(x)) # FC2 -> relu
        x = self.fc3(x) # last dense layer
        return x
    
    # Initialize model, loss function, and optimizer
# remember that cuda is using GPU with parallelism
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
net = Net().to(device)
# measures error for classification
criterion = nn.CrossEntropyLoss()
# uses ADAM optimizer to find the best weights
optmizer = optim.Adam(net.parameters(), lr=0.001)

# training function
def train(model, epochs):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0

        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            #inputs, labels = inputs.to(device), labels.to(device)
            # zero the param gradients
            optmizer.zero_grad()
            outputs = model(inputs) # predict the output with training data
            loss = criterion(outputs, labels) # see how well we did
            loss.backward() # see how to change weight to do better
            optmizer.step() # actually changes the weights
            running_loss += loss.item()
            # prints every 200 batch statistics
            if i % 200 == 199:
                print(f'Epoch [{epoch + 1}], Step [{i + 1}], Loss: {running_loss / 200:.4f}')
                running_loss = 0.0
    print('Finished Training')

train(net, 4)

Epoch [1], Step [200], Loss: 7.0393
Epoch [1], Step [400], Loss: 6.1913
Epoch [1], Step [600], Loss: 5.6102
Epoch [1], Step [800], Loss: 5.2848
Epoch [2], Step [200], Loss: 4.6941
Epoch [2], Step [400], Loss: 4.3531
Epoch [2], Step [600], Loss: 3.9506
Epoch [2], Step [800], Loss: 3.6667
Epoch [3], Step [200], Loss: 3.0192
Epoch [3], Step [400], Loss: 2.7712
Epoch [3], Step [600], Loss: 2.6210
Epoch [3], Step [800], Loss: 2.4629
Epoch [4], Step [200], Loss: 2.0221
Epoch [4], Step [400], Loss: 1.9825
Epoch [4], Step [600], Loss: 1.9873
Epoch [4], Step [800], Loss: 1.8426
Finished Training


In [27]:
# Evaluate function
def evaluate(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in train_loader:
            images, labels = data
            #images, labels = images.to(device), labels.to(device).view(-1)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy of the network: { 100 * correct / total:.2f}%')

evaluate(net)

Accuracy of the network: 57.38%
